<a href="https://colab.research.google.com/github/Armeldjogo/NLP/blob/main/Exam_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Armel Nsiangani

Student ID#: 001-99-1988


NLP - Exam 1

In [ ]:
# Downloading packages
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() 
from sklearn.feature_extraction.text import CountVectorizer # Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
from sklearn.naive_bayes import MultinomialNB # Bayes
from sklearn.ensemble import RandomForestClassifier # RF
from sklearn.svm import SVC # SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import sklearn.metrics 
from sklearn.metrics import confusion_matrix
import sklearn.metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Question 1: 

Write a generic function that takes: Classification algorithm name, vectorization method name, training set with labels as parameters (total of 3 parameters should be passed). The function should take the classification algorithm name, the vectorization method’s name, and the training set and train the desired model

In [ ]:
# Question 1
# Unzip files uploaded
import zipfile
path_to_zip_file = "/content/exam1_dataset.zip"
directory_to_extract_to = "/content/"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to) 

# Create Training data - negative dataSet
import glob, os
dirName_1 = "/content/exam1_dataset/TRAINING/negative/"
dirName_2 = "/content/exam1_dataset/TRAINING/positive/"
dirName_3 = "/content/exam1_dataset/UNLABELED/"

def returnDataInArray(dirName):
  os.chdir(dirName)
  filesName = []
  for namefile in glob.glob("*.txt"):
    filesName.append(namefile)

  reviews = []
  for i in range(0,len(filesName)):
    with open(dirName + filesName[i]) as openfile:
      data = openfile.read()
      reviews.append(data) 
  return reviews,filesName


negReviews,neg_fileName = returnDataInArray(dirName_1)
posReviews,pos_fileName = returnDataInArray(dirName_2)
unlabeledReviews,ulb_fileName = returnDataInArray(dirName_3)

# Lets merge neg & pos reviews by altenating b/w the sets 
mergedReviews = []
targetReviews = []
for i in range(0,len(negReviews)):
    mergedReviews.append(negReviews[i])
    mergedReviews.append(posReviews[i])
    targetReviews.append(0)
    targetReviews.append(1)


# Function to return models
def createModels(classAlgName,vectorMethod,trainingSet):
  # create model
  model = make_pipeline(vectorMethod, classAlgName)
  # fit model
  model.fit(trainingSet['data'], trainingSet[['target']].values)
  return model

Question 2: Using the function from question 1 to build the following models:

a) Model a: Naive Bayes, Vectorizer: TFIDF and Bag of Words, Training set should be 75% of the provided dataset. Leaving the remaining 25% for testing.

b) Model b: RandomForest, Vectorizer: TFIDF and Bag of Words, Training set should be 70% of the provided dataset. Leaving the remaining 30% for testing.

c) Model c: Support Vector Machines (SVC in sklearn), Vectorizer: TFIDF and Bag of Words, Training set should be 60% of the provided dataset. Leaving the remaining 40% for testing.

In [ ]:
# Question 2
from numpy.random import seed
seed(12345)

# DataFrame with merged data
dF = pd.DataFrame({"data": mergedReviews,"target": targetReviews})

# Bayes
Btrain, Btest = train_test_split(dF, train_size= 0.75, test_size = 0.25)

modelBayes_TDIDF = createModels(MultinomialNB(),TfidfVectorizer(),Btrain)
modelBayes_BOW = createModels(MultinomialNB(),CountVectorizer(),Btrain) 

# RF
RFtrain, RFtest = train_test_split(dF, train_size= 0.70, test_size = 0.30)

modelRF_TDIDF = createModels(RandomForestClassifier(),TfidfVectorizer(),RFtrain)
modelRF_BOW = createModels(RandomForestClassifier(),CountVectorizer(),RFtrain)

# SVC
SVCtrain, SVCtest = train_test_split(dF, train_size= 0.60, test_size = 0.40)

modelSVC_TDIDF = createModels(SVC(),TfidfVectorizer(),SVCtrain)
modelSVC_BOW = createModels(SVC(),CountVectorizer(),SVCtrain)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using 

Question 3:

Using the models from Question 2, evaluate each model with its
respective training set (for model a, that set is 25% of the data, for model b, that set is 30% of
the data, and for model c that set is 40% of the data. Be careful to not mix up the evaluation
sets. With the predictions on the test set and show the following metrics: Accuracy, Precision,
Recall, and Macro F1-score

In [ ]:
# Question 3
# Bayes
labels_B1 = modelBayes_TDIDF.predict(Btest['data'])
labels_B2 = modelBayes_BOW.predict(Btest['data'])
mat_B1 = confusion_matrix(Btest[['target']], labels_B1)
mat_B2 = confusion_matrix(Btest[['target']], labels_B2)

print('Results using Bayes & TD-IDF')
print('Accuracy:', sklearn.metrics.accuracy_score(Btest[['target']],labels_B1))
print('Precision:', sklearn.metrics.precision_score(Btest[['target']],labels_B1))
print('Recall:', sklearn.metrics.recall_score(Btest[['target']],labels_B1))
print('F1 Score:', sklearn.metrics.f1_score(labels_B1, Btest[['target']], average='macro'))

print()
print('Results using Bayes & BOW')
print('Accuracy:', sklearn.metrics.accuracy_score(Btest[['target']],labels_B2))
print('Precision:', sklearn.metrics.precision_score(Btest[['target']],labels_B2))
print('Recall:', sklearn.metrics.recall_score(Btest[['target']],labels_B2))
print('F1 Score:', sklearn.metrics.f1_score(labels_B2, Btest[['target']], average='macro'))


Results using Bayes & TD-IF
Accuracy: 0.86128
Precision: 0.8902564102564102
Recall: 0.8266666666666667
F1 Score: 0.8611711581880195

Results using Bayes & BOW
Accuracy: 0.85024
Precision: 0.876786929884275
Recall: 0.8177777777777778
F1 Score: 0.8501393166393827


In [ ]:
# Random Forest
labels_RF1 = modelRF_TDIDF.predict(RFtest['data'])
labels_RF2 = modelRF_BOW.predict(RFtest['data'])
mat_RF1 = confusion_matrix(RFtest[['target']], labels_RF1)
mat_RF2 = confusion_matrix(RFtest[['target']], labels_RF2)

print('Results using Random Forest & TD-IDF')
print('Accuracy:', sklearn.metrics.accuracy_score(RFtest[['target']],labels_RF1))
print('Precision:', sklearn.metrics.precision_score(RFtest[['target']],labels_RF1))
print('Recall:', sklearn.metrics.recall_score(RFtest[['target']],labels_RF1))
print('F1 Score:', sklearn.metrics.f1_score(labels_RF1, RFtest[['target']], average='macro'))

print()
print('Results using Random Forest & BOW')
print('Accuracy:', sklearn.metrics.accuracy_score(RFtest[['target']],labels_RF2))
print('Precision:', sklearn.metrics.precision_score(RFtest[['target']],labels_RF2))
print('Recall:', sklearn.metrics.recall_score(RFtest[['target']],labels_RF2))
print('F1 Score:', sklearn.metrics.f1_score(labels_RF2, RFtest[['target']], average='macro'))

Results using Random Forest & TD-IF
Accuracy: 0.8317333333333333
Precision: 0.8372410032715376
Recall: 0.8220021413276232
F1 Score: 0.8317057600050526

Results using Random Forest & BOW
Accuracy: 0.8338666666666666
Precision: 0.8304140127388535
Recall: 0.8375267665952891
F1 Score: 0.8338666194109494


In [ ]:
# SVC
labels_SVC1 = modelSVC_TDIDF.predict(SVCtest['data'])
labels_SVC2 = modelSVC_BOW.predict(SVCtest['data'])
mat_SVC1 = confusion_matrix(SVCtest[['target']], labels_SVC1)
mat_SVC2 = confusion_matrix(SVCtest[['target']], labels_SVC2)

print('Results using Random Forest & TD-IDF')
print('Accuracy:', sklearn.metrics.accuracy_score(SVCtest[['target']],labels_SVC1))
print('Precision:', sklearn.metrics.precision_score(SVCtest[['target']],labels_SVC1))
print('Recall:', sklearn.metrics.recall_score(SVCtest[['target']],labels_SVC1))
print('F1 Score:', sklearn.metrics.f1_score(labels_SVC1, SVCtest[['target']], average='macro'))

print()
print('Results using Random Forest & BOW')
print('Accuracy:', sklearn.metrics.accuracy_score(SVCtest[['target']],labels_SVC2))
print('Precision:', sklearn.metrics.precision_score(SVCtest[['target']],labels_SVC2))
print('Recall:', sklearn.metrics.recall_score(SVCtest[['target']],labels_SVC2))
print('F1 Score:', sklearn.metrics.f1_score(labels_SVC2, SVCtest[['target']], average='macro'))

Results using Random Forest & TD-IF
Accuracy: 0.893
Precision: 0.8829849579996093
Recall: 0.905630134241635
F1 Score: 0.8929870514332234

Results using Random Forest & BOW
Accuracy: 0.8542
Precision: 0.8332390115072628
Recall: 0.884992987377279
F1 Score: 0.85407557900168


a) What model performs the best?

The best performing model is SVC performed with TD-IDF. This is because this model has the highest accuracy, recall and f1_score. In addition, it has the second largest precision score.

b) Why is it important not to mix up the testing sets between different models?

Mixing up testing sets between different models can lead to misleading results affecting the prediction ability of the model. It is important to separate the testing sets between models as it can lead to unexpected dependancy or bias.


c) Display in a single sorted dataframe (model name, training %, test %, accuracy,
precision, recall, F1-score) all performance metrics, sorted by accuracy in descending
manner.

In [ ]:
model_name = ["SVC + TD-IDF","Bayes + TD-IDF","SVC + BOW","Bayes + BOW","RF + BOW","RF + TD-IDF"]
training_perc = [60,75,60,75,70,70]
test_perc = [40,25,40,25,30,30]
accuracy = [sklearn.metrics.accuracy_score(SVCtest[['target']],labels_SVC1),
            sklearn.metrics.accuracy_score(Btest[['target']],labels_B1),
            sklearn.metrics.accuracy_score(SVCtest[['target']],labels_SVC2),
            sklearn.metrics.accuracy_score(Btest[['target']],labels_B2),
            sklearn.metrics.accuracy_score(RFtest[['target']],labels_RF2),
            sklearn.metrics.accuracy_score(RFtest[['target']],labels_RF1)]

precision = [sklearn.metrics.precision_score(SVCtest[['target']],labels_SVC1),
            sklearn.metrics.precision_score(Btest[['target']],labels_B1),
            sklearn.metrics.precision_score(SVCtest[['target']],labels_SVC2),
            sklearn.metrics.precision_score(Btest[['target']],labels_B2),
            sklearn.metrics.precision_score(RFtest[['target']],labels_RF2),
            sklearn.metrics.precision_score(RFtest[['target']],labels_RF1)]

recall = [sklearn.metrics.recall_score(SVCtest[['target']],labels_SVC1),
            sklearn.metrics.recall_score(Btest[['target']],labels_B1),
            sklearn.metrics.recall_score(SVCtest[['target']],labels_SVC2),
            sklearn.metrics.recall_score(Btest[['target']],labels_B2),
            sklearn.metrics.recall_score(RFtest[['target']],labels_RF2),
            sklearn.metrics.recall_score(RFtest[['target']],labels_RF1)]

f1_score = [sklearn.metrics.f1_score(SVCtest[['target']],labels_SVC1),
            sklearn.metrics.f1_score(Btest[['target']],labels_B1),
            sklearn.metrics.f1_score(SVCtest[['target']],labels_SVC2),
            sklearn.metrics.f1_score(Btest[['target']],labels_B2),
            sklearn.metrics.f1_score(RFtest[['target']],labels_RF2),
            sklearn.metrics.f1_score(RFtest[['target']],labels_RF1)]

# Data Frame + scores
dataFrame_with_scores = pd.DataFrame({"model name": model_name,"training %": training_perc, "test %": test_perc,
                                      "accuracy" : accuracy, "precision": precision,
                                      "recall":recall, "f1_score":f1_score})
print(dataFrame_with_scores)

       model name  training %  test %  accuracy  precision    recall  f1_score
0    SVC + TD-IDF          60      40  0.893000   0.882985  0.905630  0.894164
1  Bayes + TD-IDF          75      25  0.861280   0.890256  0.826667  0.857284
2       SVC + BOW          60      40  0.854200   0.833239  0.884993  0.858337
3     Bayes + BOW          75      25  0.850240   0.876787  0.817778  0.846255
4        RF + BOW          70      30  0.833867   0.830414  0.837527  0.833955
5     RF + TD-IDF          70      30  0.831733   0.837241  0.822002  0.829552


Question 4:

Using the documents in the folder named UNLABELED, please use
your best performing trained model from question 3 to predict their labels. Please do this
individually for each document.

In [ ]:
# Question 4
# Using SVC + TD-IDF to predict
def predict_category(s, model = modelSVC_TDIDF):
    pred = model.predict([s])
    if pred[0] == 0:
      f_pred = 'negative'
    else:
      f_pred = 'positive'
    return f_pred,pred

#unlabeledReviews,ulb_fileName
predictions = []
score = []
for k in range(0,len(unlabeledReviews)):
  x,scr = predict_category(unlabeledReviews[k])
  predictions.append(x)
  score.append(scr)


dataFrame_pred = pd.DataFrame({"Doc name": ulb_fileName, "Predicted labeled": predictions})
print(dataFrame_pred)


       Doc name Predicted labeled
0   46278_0.txt          negative
1   36517_0.txt          negative
2   24221_0.txt          negative
3   46705_0.txt          negative
4   37154_0.txt          negative
5       0_0.txt          positive
6   36022_0.txt          negative
7   36149_0.txt          negative
8   35991_0.txt          negative
9   49990_0.txt          negative
10  35968_0.txt          negative


Own opinion: 


I have to admit, that I was a little surprised when the model predicted that all reviews except for the first one was negative. However, after reading the review, I believe the model was mostly right. Two mistakes may be with the prediction of review 0_0.txt and 46278_0.txt. I think that 0_0.txt is negative, and 46278_0.txt is positive, but I may be wrong.

In [ ]:
# Question 5
def matrixSimCos(all_docs):
  vect = TfidfVectorizer(stop_words = 'english')
  tfidf_row = vect.fit_transform(all_docs)                                                                                                                                                                                                
  tfidf = vect.transform(all_docs)                                                                                                                                                                                                                       
  cos_similarity = sklearn.metrics.pairwise.cosine_similarity(tfidf,tfidf_row)
  return cos_similarity

alldocs = negReviews + posReviews
rslt = matrixSimCos(alldocs)

sns.heatmap(rslt, square=True, annot=False, cbar=True,
            xticklabels= "Negative to Positive reviews", yticklabels= "Negative to Positive reviews")

In [ ]:
# Question 6
def findMostSim(m,n):
  array = np.around(m, decimals=3)
  rArray = array.reshape(-1)
  uN_Array = np.unique(rArray)
  listVal = np.flip(uN_Array)
  listVal = np.delete(listVal, np.where(listVal == 1))
  valsOfInterest = listVal[0:n]
  locVals = []
  for i in range(0,len(valsOfInterest)):
    rslt = np.where(array == valsOfInterest[i])
    locVals.append(rslt[0].tolist())

  return locVals,valsOfInterest

m = rslt
n = 50
filesName = neg_fileName + pos_fileName
loc,vals = findMostSim(m,n)
print("The",n,"most similar works are in position: ")
print('-----------------------------------------')
indx = 0
for pos in loc:
  print(filesName[pos[0]],'and',filesName[pos[1]], 'Similarity = ', vals[indx])
  indx += 1

Do all similar documents belong to same class? 

No, all similar documents do not belong to same class. Exisiting similarities were found between Negative and Positive reviews. This is probably due to people who somwhat liked or somewhat disliked the movie. Since, their sentiments were not strong enough to fully put in one category, there will be a high probability of similarity.

Question 7:


Using Spacy’s part of speech tagger, process all sentences (hint: don’t
forget to split the reviews) and count how many NOUN and VERB tags are found in all the
movies review (TRAINING folder) separating them by label. 

In [ ]:
# Question 7
import spacy
nlp = spacy.load("en_core_web_sm")
def foundCount(docSet):
  verb_count = 0
  noun_count = 0
  for kk in range(0,len(docSet)):
    doc_1 = nlp(docSet[kk])
    for token in doc_1:
      if token.pos_ == 'NOUN':
        noun_count += 1
      elif token.pos_ == 'VERB':
        verb_count += 1
  return verb_count,noun_count

neg_verb, neg_noun = foundCount(negReviews)
pos_verb, pos_noun = foundCount(posReviews)

print("In Negative reviews, there are",neg_noun, "nouns and", neg_verb, "verbs")
print("In Positive reviews, there are",pos_noun, "nouns and", pos_verb, "verbs")

In Negative reviews, there are 528475 nouns and 355559 verbs
In Positive reviews, there are 542978 nouns and 342125 verbs


Opinion:

Yes, there is a difference between the number of nouns and verbs in Negative vs. Positive review. I notice that there is less nouns in Negative than Positive reviews, but less verbs in Positive then Negative reviews. However, in both cases, the difference in word counts is less than 10%, so I would not say that there is a huge disparity. This could mean that regardless of their impressions movies goers tend to express both positive and negative opinions equally.

Question 8:

Using the results from the PoS process in question 7, count how many
different PUNCT tags are found and their respective counts from the full dataset provided (both
negative and positives together).

In [ ]:
# Question 8
import re
# Using Spacy
def foundCount(docSet):
  punct_count = 0
  for kk in range(0,len(docSet)):
    doc_1 = nlp(docSet[kk])
    for token in doc_1:
      if token.pos_ == 'PUNCT':
        punct_count += 1
  return punct_count
spacy_punct_count = foundCount(mergedReviews)


In [ ]:
# Using regex
def punctCount(docSet):
  data = docSet
  punct_count = 0
  for i in range(0,len(data)):
    m = re.findall(r'[.?\-/",!]+', data[i])
    punct_count = punct_count + len(m)
  return punct_count
regex_punct_count = punctCount(mergedReviews)


In [ ]:
print('The number of PUNCT tags using Spacy is:',spacy_punct_count)
print('The number of PUNCT tags using regex is:',regex_punct_count)

The number of PUNCT tags using Spacy is: 834967
The number of PUNCT tags using regex is: 829922


Opinion:

I was not able to get the same count using Spacy and Regex. This is probably because what I have defined as "punctuation" in my regex is not the same as the characters defined as "punctuation" by Spacy. However, if I used the same set of punctuation as Spacy, I would have had the same count.